In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline





In [2]:
data = pd.read_csv("C:\\Users\\SUNDAR\\Desktop\\codes\\major project\\spam.csv",encoding = 'latin1')

data['lab'] = data["Category"].replace({"ham":0,"spam":1})
data.head()


,Category,Message,lab
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [3]:
X = data.Message
Y = data.Category
le = LabelEncoder()#converts categorical variables to numverical format
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.20)

In [5]:
#spliting into words using tokenization
#tokens- words
max_words = 1000
max_len  = 150
tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen = max_len)


In [6]:
#Embedding layer - used for neural networks on text data.It input data be integer
#each word is encoded as unique integer...it can be used to load a pre-trained word embedding model- a type of transfer learning
def rnn():
    inputs = Input(name = 'inputs',shape = [max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name = 'FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name = 'out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs = inputs, outputs = layer)
    return model

In [7]:
model = rnn()
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(),metrics = ['accuracy'])


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [8]:
#calllback is a set of function to be applied at given stages of the training procedure.
#can use callback to get a view on internal states and statistics of the model during training
model.fit(sequences_matrix,Y_train,batch_size=128,epochs = 10,
          validation_split=0.2, callbacks=[EarlyStopping(monitor = 'val_loss',min_delta=0.0001)])

Epoch 1/10
28/28 [==============================] - 37s 761ms/step - loss: 0.3947 - accuracy: 0.8640 - val_loss: 0.2194 - val_accuracy: 0.9092
Epoch 2/10
28/28 [==============================] - 14s 515ms/step - loss: 0.1553 - accuracy: 0.9565 - val_loss: 0.1077 - val_accuracy: 0.9742
Epoch 3/10
28/28 [==============================] - 14s 499ms/step - loss: 0.0777 - accuracy: 0.9806 - val_loss: 0.0800 - val_accuracy: 0.9787
Epoch 4/10
28/28 [==============================] - 12s 443ms/step - loss: 0.0530 - accuracy: 0.9851 - val_loss: 0.0649 - val_accuracy: 0.9821
Epoch 5/10
28/28 [==============================] - 7s 266ms/step - loss: 0.0467 - accuracy: 0.9877 - val_loss: 0.0638 - val_accuracy: 0.9809
Epoch 6/10
28/28 [==============================] - 7s 266ms/step - loss: 0.0371 - accuracy: 0.9896 - val_loss: 0.0590 - val_accuracy: 0.9776
Epoch 7/10
28/28 [==============================] - 8s 301ms/step - loss: 0.0290 - accuracy: 0.9924 - val_loss: 0.0959 - val_accuracy: 0.9776


In [9]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen = max_len) 

In [10]:
acc = model.evaluate(test_sequences_matrix,Y_test)

35/35 [==============================] - 2s 45ms/step - loss: 0.0745 - accuracy: 0.9821


In [55]:
test_content = [input()]

textx = tok.texts_to_sequences(test_content)
textx = sequence.pad_sequences(textx,maxlen=max_len)




In [56]:
pred = model.predict(textx)
print(pred)

1/1 [==============================] - 0s 91ms/step
[[0.99921185]]


In [57]:
if pred > [[0.5]]:
    print("This is a Spam mail")
else:
    print("This is not a Spam mail")

This is a Spam mail


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

from flask import Flask, render_template, request



app = Flask(__name__)

data = pd.read_csv("C:\\Users\\SUNDAR\\Desktop\\codes\\major project\\spam.csv",encoding = 'latin1')

data['lab'] = data["Category"].replace({"ham":0,"spam":1})
data.head()

X = data.Message
Y = data.Category
le = LabelEncoder()#converts categorical variables to numverical format
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.20)

#spliting into words using tokenization
#tokens- words
max_words = 1000
max_len  = 150
tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen = max_len)

#Embedding layer - used for neural networks on text data.It input data be integer
#each word is encoded as unique integer...it can be used to load a pre-trained word embedding model- a type of transfer learning
def rnn():
    inputs = Input(name = 'inputs',shape = [max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name = 'FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name = 'out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs = inputs, outputs = layer)
    return model

model = rnn()
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(),metrics = ['accuracy'])

#calllback is a set of function to be applied at given stages of the training procedure.
#can use callback to get a view on internal states and statistics of the model during training
model.fit(sequences_matrix,Y_train,batch_size=128,epochs = 10,
          validation_split=0.2, callbacks=[EarlyStopping(monitor = 'val_loss',min_delta=0.0001)])

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen = max_len) 

acc = model.evaluate(test_sequences_matrix,Y_test)

test_content = input()
textx = tok.texts_to_sequences(test_content)
textx = sequence.pad_sequences(textx,maxlen=max_len)
@app.route('/')
def home():
    return render_template('index.html')  # Create an HTML file for the form

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        message = request.form['message']
        textx = tok.texts_to_sequences([message])
        textx = sequence.pad_sequences(textx, maxlen=max_len)
        pred = model.predict(textx)

        if pred > 0.05:
            result = "This is a Spam mail"
        else:
            result = "This is not a Spam mail"

        return render_template('result.html', result=result)

if __name__ == '__main__':
    app.run(debug=True) 
